# Evaluate Plots V2

test pair에 대한 iteration을 수행한다.

1. pickle을 통해서 모든 아이템을 다 읽는다.
2. 현재 source sequence를 읽는다.
3. scene similarity를 읽는다.
- mistral로 계산한 inclusion ratio
- 직접 계산한 entity similarity

4. action sequence similarity를 계산한다.
- agent4 필터 통과와 상관없이 플롯
- agent4 필터를 통과하면 플롯

5. activity taxnomy similarity를 계산한다.
- agent4 필터 통과와 상관없이 플롯
- agent4 필터를 통과하면 플롯


다음의 pandas colum을 구성한다. (568x13 정도 되나?)

없는 곳은 None표시를 한다.

source idx, target idx, 
augmentation_id, inclusion_ratio, entity_similarity
goal_category, goal_description
core activity_gt, core activity_inf
sequence sim, taxonomy sim
sequence bool, taxonomy bool

In [36]:
%matplotlib qt
import sys
import os
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import mplcursors
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from bert_score import score

import pickle

sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))
from util import util_constants
from util import util_funcs
import f4_evaluate.evaluate_scene as evaluate_scene
import f1_init.database_init as database_init
import f1_init.agent_init as agent_init
import f1_init.constants_init as constants_init

#Computing similarity
import json
from sentence_transformers import SentenceTransformer
from bert_score import score
import numpy as np
from numpy.linalg import norm
import ast
import pandas as pd

#PATHS
PATH_CURR_FOLDER = os.path.abspath('') 
PATH_DATA_ANNOTATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_annotation'))
PATH_DATA_INPUT_OUTPUT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output'))
PATH_SOURCE_TARGET_INPUT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/input/source_target_video_list.pkl'))
TEST_SPATIAL_ANNOTATION_PATH_MANUAL = PATH_DATA_ANNOTATION + '/spatial_testset/manual'

#PAIRSIM DATA
PATH_PAIRSIM = PATH_DATA_ANNOTATION + '/spatial_pairsim_result'

#AUGMENTED DATA PATH
TEST_SPATIAL_ANNOTATION_PATH_SEMI = PATH_DATA_ANNOTATION + '/spatial_testset/semi'    
TEST_SPATIAL_ANNOTATION_V2_PATH = PATH_DATA_ANNOTATION + '/spatial_augmentation/TESTSET_Augmented_Data_v2/'

#BASELINE RESULT PATHS
PATH_BASELINE_RAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-rag/'))
PATH_BASELINE_NORAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-norag/'))
PATH_BASELINE_1DIRECT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1direct/'))
PATH_BASELINE_1GOALMEDIATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1goalmediation/'))




QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'


In [ ]:
def load_file(path):
    try:          
         with open(path, "rb") as f:
             data = pickle.load(f)
             return data
    except:
         return None
    
#=====================================
# SIMILARITIES
#=====================================
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if norm(vec1) == 0 or norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

def safe_parse_sequence(seq):
    if seq is None:
        return []
    if isinstance(seq, list):
        return seq
    try:
        return ast.literal_eval(seq)
    except (ValueError, SyntaxError):
        print("taxonomy safe parse value error")
        return []

def safe_parse_taxonomy(tax):
    if tax is None:
        return {}
    if isinstance(tax, dict):
        return tax
    try:
        return json.loads(tax)
    except (json.JSONDecodeError, TypeError):
        return {}

def compute_similarities(entry, embed_model):
    # --- Parse sequences ---
    source_seq = safe_parse_sequence(entry.get('source_sequence'))
    target_seq = safe_parse_sequence(entry.get('target_sequence'))

    if not source_seq or not target_seq:
        seq_similarity = 0.0
    else:
        source_seq_str = ' '.join(source_seq)
        target_seq_str = ' '.join(target_seq)
        source_seq_emb = embed_model.encode(source_seq_str)
        target_seq_emb = embed_model.encode(target_seq_str)
        seq_similarity = cosine_similarity(source_seq_emb, target_seq_emb)

    # --- Parse taxonomies ---
    source_tax = safe_parse_taxonomy(entry.get('source_taxonomy'))
    target_tax = safe_parse_taxonomy(entry.get('target_taxonomy'))

    if not source_tax or not target_tax:
        tax_similarity = 0.0
    else:
        source_tax_str = ' '.join(f"{k}: {v}" for k, v in source_tax.items())
        target_tax_str = ' '.join(f"{k}: {v}" for k, v in target_tax.items())
        source_tax_emb = embed_model.encode(source_tax_str)
        target_tax_emb = embed_model.encode(target_tax_str)
        tax_similarity = cosine_similarity(source_tax_emb, target_tax_emb)

    return seq_similarity, tax_similarity

## Read Results

read a list of 568 pickle sets in 8 elements for 8 levels in augmentation

make dictionary
-fill every information
-fill every calculatable information
-save as df

In [173]:

source_list = [] # [0,0,0,0,1,1,1,1,...,283] -> [0,1,2,3,..71,0,1,2,3,4..71,...71]
target_list = [] # [augno_0: 55, aug33_0: 33,aug67_0: 28,aug100_0: 22,...] -> [55,26,22,...]
baselines = ["rag","norag","1direct","1goalmediation"]
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')


def make_singlebaseline_list(baseline_result_path:str, baseline:str):
    '''
    func: make single baseline list

    '''
    baseline_result_path = baseline_result_path + "/"
    baseline_results = []
    augmodes = [30, 40, 50, 60, 70, 80, 90, 100]
    length = 71 * 8 #8levels, 8files

    for i in range(length):
    # for i in range(568):
        # print(i)
        dict = {
            "idx":None,"source_idx":None, "target_idx":None, "source_uid": None, "target_uid": None, "baseline": None, "augmode": None, 
            "source_sequence": None, "source_scene_graph": None, "target_scene_graph": None, 
            # "core_activity": None, "source_taxonomy": None, "common_taxonomy": None, 
            # "target_taxonomy": None, "target_sequence": None, 
            # "sequence_boolean": None, "taxonomy_boolean": None, 
            "inclusion_ratio":None, #"pairwise_similarity": None,
            #"sequence_similarity_sbert": None, "taxonomy_similarity_sbert":None
            }

        prefix = f"pair{i}_"
        augmode = i%len(augmodes)
        # FILL RAW DATA
        sourceinfo_dict = load_file(baseline_result_path + prefix + "sourceinfo.pkl")
        targetinfo_dict = load_file(baseline_result_path + prefix + "targetinfo.pkl")
        dict["idx"] = i
        dict["source_idx"] = sourceinfo_dict['source_idx']
        dict["target_idx"] = targetinfo_dict['target_idx']
        dict["source_uid"] = sourceinfo_dict['source_uid']
        dict["target_uid"] = targetinfo_dict['target_uid']
        dict["baseline"] = baseline
        dict["augmode"] = augmode


        dict['source_sequence'] =sourceinfo_dict['source_action_sequence']
        dict['source_sequence'] = dict['source_sequence'].strip('"').split(', ') #make into list

        dict['source_scene_graph'] =sourceinfo_dict['source_scene_graph']
        dict['target_scene_graph'] =targetinfo_dict['target_scene_graph']

        
        # based on baselines     
        if baseline == "1direct":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
        if baseline == "1goalmediation":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
            dict['core_activity'] =load_file(baseline_result_path + prefix + "agent1a.pkl")
        elif baseline == "rag" or baseline =="norag":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
            dict['core_activity'] =load_file(baseline_result_path + prefix + "agent1a.pkl")
            dict['source_taxonomy'] =load_file(baseline_result_path + prefix + "agent1b.pkl")
            dict['common_taxonomy'] =load_file(baseline_result_path + prefix + "agent2a.pkl")
            dict['target_taxonomy'] =load_file(baseline_result_path + prefix + "agent2b.pkl")


        booleans= load_file(baseline_result_path+prefix+"agent4.pkl")
        if(booleans == None):
            ## No values whatsover
            dict["sequence_boolean"] = False
            dict["taxonomy_boolean"] = False 
        else:
            values = [
                line.split(":")[1].strip()
                for line in booleans.strip().splitlines()
                if ":" in line
            ]
            # print(values)
            dict["taxonomy_boolean"] = True if values[0] == 'yes' else False
            dict["sequence_boolean"] = True if values[1] == 'yes' else False
            # print(f"{dict['taxonomy_boolean']} {dict['sequence_boolean']}")
        
        dict['inclusion_ratio'] = sourceinfo_dict['spatial_similarity']
        #dict['pairwise_similarity'] = compare_scene_graph(dict['source_scene_graph'], dict['target_scene_graph'])
        # SBERT for block level semantics

        if dict['target_sequence'] != None:
            
            # Preprocess sequece to make sequence a single string
            dict['target_sequence'] = ast.literal_eval(dict['target_sequence'])
            dict['target_sequence'] = [", ".join(dict['target_sequence'])] # to single item
            dict['source_sequence'] = [", ".join(dict['source_sequence'])]# to single item
            
            # print(dict['source_taxonomy'])
            # print(dict['target_taxonomy'])

            sequence_similarity_sbert, taxonomy_similarity_sbert = compute_similarities(dict, sbert_model)
            dict["sequence_similarity_sbert"]= sequence_similarity_sbert
            dict["taxonomy_similarity_sbert"]= taxonomy_similarity_sbert
            # print(dict["taxonomy_similarity_sbert"])

            # BERTSCORE for little more stepwise precision in similarity measurement
            # print(len(dict['target_sequence']))
            # print(len(dict['source_sequence']))
            # print(dict['target_sequence'])
            # print(dict['source_sequence'])# not inside bracket
            # print(dict['target_taxonomy'])
            # print(dict['source_taxonomy'])

            #takes very long
            # dict["sequence_b_P"], dict["sequence_b_R"], dict["sequence_b_F1"] = score(dict['target_sequence'], dict['source_sequence'], lang="en", rescale_with_baseline=True)

            #This part gives trouble
            # dict["taxonomy_b_P"], dict["taxonomy_b_R"], dict["taxonomy_b_F1"] =score(dict['target_taxonomy'], dict['source_taxonomy'], lang="en", rescale_with_baseline=True)
        baseline_results.append(dict)
    return baseline_results

baseline_rag_list = make_singlebaseline_list(PATH_BASELINE_RAG, baselines[0])
baseline_norag_list = make_singlebaseline_list(PATH_BASELINE_NORAG, baselines[1])
baseline_1direct_list = make_singlebaseline_list(PATH_BASELINE_1DIRECT, baselines[2])
baseline_1goalmediation_list = make_singlebaseline_list(PATH_BASELINE_1GOALMEDIATION, baselines[3])

# print(f"{len(baseline_rag_list)} {len(baseline_norag_list[0])}")
# print(f"{len(baseline_norag_list)} {len(baseline_norag_list[0])}")
# print(f"{len(baseline_1direct_list)} {len(baseline_1direct_list[0])}")
# print(f"{len(baseline_1goalmediation_list)} {len(baseline_1goalmediation_list[0])}")


## Perform Numerical Calculation of Results

### base results
- pairwise scene similarity
- seq2seq similarity
- seq2seq taxonomy similarity
### plot
- per scenario similarity analysis
- per augmentation, per scenario similarity analysis
- per scenario similarity vs similarity



In [174]:

def mask_nan_df(df):
    mask = df.isna().any(axis=1)
    df.loc[mask, :] = np.nan 
    
def mask_nan_df_or_condition(df, columns_to_check):
    '''
    input: columns_to_check = ["col1", "col2"]
    '''
    mask = df[columns_to_check].isna().any(axis=1)
    df.loc[mask, :] = np.nan    

def save_baseline_list(path, baseline_list):
    with open(path, 'wb') as f:
        pickle.dump(baseline_list, f)
        print(f"saved baseline list at {path}")

# turn list of dictionary into dataframe
# df_rag = pd.DataFrame(baseline_rag_list)


df_rag = pd.DataFrame(baseline_rag_list)
# mask_nan_df_or_condition(df_rag, ["target_taxonomy","target_sequence"])

df_norag = pd.DataFrame(baseline_norag_list)
# mask_nan_df_or_condition(df_norag, ["target_taxonomy","target_sequence"])

df_1direct = pd.DataFrame(baseline_1direct_list)
mask_nan_df_or_condition(df_1direct, ["target_sequence"])
# print(df_1direct.head())

df_1goalmediation = pd.DataFrame(baseline_1goalmediation_list)
mask_nan_df_or_condition(df_1goalmediation, ["target_sequence"])
# print(df_1goalmediation.head())


baseline_rag_path = 'evaluation_result_v4/rag_dict.pkl'
baseline_norag_path = 'evaluation_result_v4/norag_dict.pkl'
baseline_1direct_path = 'evaluation_result_v4/1direct_dict.pkl'
baseline_1goalmediation_list_path = 'evaluation_result_v4/1goalmediation_dict.pkl'

save_baseline_list(baseline_rag_path, baseline_rag_list)
save_baseline_list(baseline_norag_path, baseline_norag_list)
save_baseline_list(baseline_1direct_path, baseline_1direct_list)
save_baseline_list(baseline_1goalmediation_list_path, baseline_1goalmediation_list)


print(df_rag.shape[0])
print(df_norag.shape[0])
print(df_1direct.shape[0])
print(df_1goalmediation.shape[0])

print(df_rag['sequence_boolean'])
print(df_norag['sequence_boolean'])

print(df_rag['taxonomy_boolean'])
print(df_norag['taxonomy_boolean'])




saved baseline list at evaluation_result_v4/rag_dict.pkl
saved baseline list at evaluation_result_v4/norag_dict.pkl
saved baseline list at evaluation_result_v4/1direct_dict.pkl
saved baseline list at evaluation_result_v4/1goalmediation_dict.pkl
568
568
568
568
0      False
1      False
2      False
3      False
4      False
       ...  
563     True
564    False
565     True
566    False
567     True
Name: sequence_boolean, Length: 568, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
563    False
564     True
565    False
566    False
567    False
Name: sequence_boolean, Length: 568, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
563     True
564     True
565     True
566     True
567     True
Name: taxonomy_boolean, Length: 568, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
563     True
564     True
565     True
566     True
567     True
Name: taxonomy_boole

/tmp/ipykernel_3729869/871926004.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[mask, :] = np.nan
/tmp/ipykernel_3729869/871926004.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[mask, :] = np.nan
/tmp/ipykernel_3729869/871926004.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[mask, :] = np.nan
/tmp/ipykernel_3729869/871926004.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a fut

In [148]:

import pandas as pd
import matplotlib.pyplot as plt

#======================
#interactive cursor hover
#======================
def plot_taxonomy_cursor(df:pd.DataFrame, mybool:bool):
    df = df.dropna(subset=['inclusion_ratio', 'taxonomy_similarity_sbert', 'taxonomy_boolean'])
    size1 = df.shape[0]
    filtered_df = df[df['taxonomy_boolean'] == mybool]
    size2 = filtered_df.shape[0]
    print(filtered_df.shape[0])

    # Step 2: Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    sc = ax.scatter(
        filtered_df['inclusion_ratio'], 
        filtered_df['taxonomy_similarity_sbert'], 
        alpha=0.7
    )

    ax.set_xlabel('Inclusion Ratio')
    ax.set_ylabel('Taxonomy Similarity')
    ax.set_title(f'Inclusion Ratio vs. Taxonomy Similarity (Only where taxonomy_boolean={mybool}) {size2}/{size1}')

    # Polynomial fit
    N = 1
    coeffs = np.polyfit(filtered_df['inclusion_ratio'], filtered_df['taxonomy_similarity_sbert'], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['inclusion_ratio'].min(), filtered_df['inclusion_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)
    ax.plot(x_smooth, y_smooth, color='red', label=f'{N}° polynomial fit')

    # Limits and layout
    ax.set_xlim(0, 1.1)
    ax.set_ylim(0, 1.1)
    ax.grid(True)
    plt.tight_layout()

    # mplcursors for interactive tooltips
    cursor = mplcursors.cursor(sc, hover=True)
    
    @cursor.connect("add")
    def on_add(sel):
        idx = sel.index
        # Customize the tooltip content here
        sel.annotation.set_text(
            f"x: {filtered_df['inclusion_ratio'].iloc[idx]:.2f}\n"
            f"y: {filtered_df['taxonomy_similarity_sbert'].iloc[idx]:.2f}\n"
            f"info: {filtered_df.get('source_uid', pd.Series(['N/A'] * size2)).iloc[idx]}"
        )

    plt.show()    

def plot_sequence_cursor(df:pd.DataFrame, mybool:bool):
    df = df.dropna(subset=['inclusion_ratio', 'sequence_similarity_sbert', 'sequence_boolean'])
    filtered_df = df[df['sequence_boolean'] == mybool]
    size1 = df.shape[0]
    size2 = filtered_df.shape[0]
    print(size1, size2)

    # Step 2: Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    sc = ax.scatter(
        filtered_df['inclusion_ratio'], 
        filtered_df['sequence_similarity_sbert'], 
        alpha=0.7
    )

    ax.set_xlabel('Inclusion Ratio')
    ax.set_ylabel('Sequence Similarity')
    ax.set_title(f'Inclusion Ratio vs. Sequence Similarity (bool={mybool}) {size2}/{size1}')

    # Polynomial fit
    N = 1
    coeffs = np.polyfit(filtered_df['inclusion_ratio'], filtered_df['sequence_similarity_sbert'], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['inclusion_ratio'].min(), filtered_df['inclusion_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)
    ax.plot(x_smooth, y_smooth, color='red', label=f'{N}° polynomial fit')

    # Limits and layout
    ax.set_xlim(0, 1.1)
    ax.set_ylim(0, 1.1)
    ax.grid(True)
    plt.tight_layout()

    # mplcursors for interactive tooltips
    cursor = mplcursors.cursor(sc, hover=True)
    
    @cursor.connect("add")
    def on_add(sel):
        idx = sel.index
        # Customize the tooltip content here
        sel.annotation.set_text(
            f"x: {filtered_df['inclusion_ratio'].iloc[idx]:.2f}\n"
            f"y: {filtered_df['sequence_similarity_sbert'].iloc[idx]:.2f}\n"
            f"info: {filtered_df.get('source_uid', pd.Series(['N/A'] * size2)).iloc[idx]}"
        )
 

    plt.show()    

#======================
#non interactive
#======================
def plot_taxonomy(df):
    df = df.dropna(subset=['inclusion_ratio', 'taxonomy_similarity_sbert', 'taxonomy_boolean'])
    size1 = df.shape[0]
    # df = df.dropna()
    mybool = True
    filtered_df = df[df['taxonomy_boolean'] == mybool]
    size2 = filtered_df.shape[0]
    print(filtered_df.shape[0])
    # Step 2: Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(filtered_df['inclusion_ratio'], filtered_df['taxonomy_similarity_sbert'], alpha=0.7)
    plt.xlabel('Inclusion Ratio')
    plt.ylabel('Taxonomy Similarity')
    plt.title(f'Inclusion Ratio vs. Taxonomy Similarity (Only where taxonomy_boolean={mybool}) {size2}/{size1}')


    #Generate smooth x values for curve plotting
    N=1
    coeffs = np.polyfit(filtered_df['inclusion_ratio'], filtered_df['taxonomy_similarity_sbert'], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['inclusion_ratio'].min(), filtered_df['inclusion_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)
    plt.plot(x_smooth, y_smooth, color='red', label=f'{N}° polynomial fit')


    plt.xlim(0, 1.1)
    plt.ylim(0, 1.1)  

    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_sequence(df):
    df = df.dropna(subset=['inclusion_ratio', 'sequence_similarity_sbert', 'sequence_boolean'])
    size1 = df.shape[0]
    mybool = True
    filtered_df = df[df['sequence_boolean'] == mybool]
    size2 = filtered_df.shape[0]

    # filtered_df = df

    # Step 2: Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(filtered_df['inclusion_ratio'], filtered_df['sequence_similarity_sbert'], alpha=0.7)
    plt.xlabel('Inclusion Ratio')
    plt.ylabel('Sequence Similarity')
    plt.title(f'Inclusion Ratio vs. Sequence Similarity (Only where sequence_boolean={mybool} {size2}/{size1})')

    # Generate smooth x values for curve plotting
    N=1
    coeffs = np.polyfit(filtered_df['inclusion_ratio'], filtered_df['sequence_similarity_sbert'], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['inclusion_ratio'].min(), filtered_df['inclusion_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)
    plt.plot(x_smooth, y_smooth, color='red', label=f'{N}° polynomial fit')

    plt.xlim(0, 1.1)
    plt.ylim(0, 1.1)  

    plt.grid(True)
    plt.tight_layout()
    plt.show()    


plot_taxonomy_cursor(df_rag, True)
plot_taxonomy_cursor(df_rag, False)
plot_taxonomy_cursor(df_norag, True)
plot_taxonomy_cursor(df_norag, False)
# plot_taxonomy_cursor(df_norag)


# plot_sequence_cursor(df_rag, True)
# plot_sequence_cursor(df_norag, True)
# plot_sequence_cursor(df_1direct, False)
# plot_sequence_cursor(df_1goalmediation, True)


202
353
177
348


In [11]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mplcursors

# Dummy DataFrame
df = pd.DataFrame({
    'inclusion_ratio': np.random.rand(10),
    'sequence_similarity_sbert': np.random.rand(10),
    'taxonomy_boolean': [True] * 10,
    'source_uid': [f"Item {i}" for i in range(40, 50)]
})

# plot_sequence_cursor(df)

In [188]:

def filter_taxdf(df:dict):
    size0 = df.shape[0]
    df = df.dropna(subset=['inclusion_ratio', 'taxonomy_similarity_sbert', 'taxonomy_boolean', 'sequence_similarity_sbert', 'sequence_boolean'])
    size1 = df.shape[0]
    filtered_df = df[df['taxonomy_boolean'] == True]
    size2 = filtered_df.shape[0]
    filtered_df = filtered_df[filtered_df['sequence_boolean'] == True]
    size3 = filtered_df.shape[0]

    print(f"{size0} {size1} {size2} {size3}")
    return filtered_df

filtered_df_rag = filter_taxdf(df_rag)
filtered_df_norag = filter_taxdf(df_norag)

def filter_by_sbert(df:dict, sbertitem:str, min:float, max:float):
    filtered_df = df[df[sbertitem] > min]
    filtered_df = filtered_df[filtered_df[sbertitem] <= max]
    size = filtered_df.shape[0]
    # print(f"{size} : {list(filtered_df['source_uid'])}")
    print("idx")
    filtered_df = filtered_df.sort_values(by='idx')
    for i in range(size):
        print(f"idx: {filtered_df.iloc[i]['idx']}, {filtered_df.iloc[i]['inclusion_ratio']:.2f}, source_uid: {filtered_df.iloc[i]['source_uid']}, target_uid: {filtered_df.iloc[i]['target_uid']}")

    print("inclusion_ratio")
    filtered_df = filtered_df.sort_values(by='inclusion_ratio')
    for i in range(size):
        print(f"idx: {filtered_df.iloc[i]['idx']}, {filtered_df.iloc[i]['inclusion_ratio']:.2f}, source_uid: {filtered_df.iloc[i]['source_uid']}, target_uid: {filtered_df.iloc[i]['target_uid']}")

    return filtered_df

def get_common_dict(dict1:dict, dict2:dict, k:str):
    '''
    k:keyname
    '''
    return {k: dict1[k] for k in dict1 if k in dict2 and dict1[k] == dict2[k]}



# find passed true rags and lookat same index
common_idx = set(filtered_df_rag['idx']) & set(filtered_df_norag['idx'])
df_common = filtered_df_rag[filtered_df_rag['idx'].isin(common_idx)]
print(df_common.shape[0])
df_common1 = filter_by_sbert(df_common, 'taxonomy_similarity_sbert', 0.95, 1.0)
df_common2 = filter_by_sbert(df_common, 'taxonomy_similarity_sbert', 0.90, 0.95)


# # for 1.0 both seq and taxonomy survives
# print("sbert 1.0")
# filtered_df_rag_tax = filter_by_sbert(df_rag, 'taxonomy_similarity_sbert', 0.95, 1.0)
# filtered_df_norag_tax = filter_by_sbert(df_norag, 'taxonomy_similarity_sbert', 0.95,1.0)
# filtered_df_rag_seq = filter_by_sbert(df_rag, 'sequence_similarity_sbert', 0.95,1.0)
# filtered_df_norag_seq = filter_by_sbert(df_norag, 'sequence_similarity_sbert', 0.95,1.0)

# common_rag = list(set(list(filtered_df_rag_tax['source_uid'])) & set(list(filtered_df_rag_seq['source_uid'])))
# common_norag = list(set(list(filtered_df_norag_tax['source_uid'])) & set(list(filtered_df_norag_seq['source_uid'])))
# more_common_list1 = list(set(common_rag) & set(common_norag))
# print(f"{len(common_rag)} {common_rag}")
# print(f"{len(common_norag)} {common_norag}")
# print(f"{len(more_common_list1)} {more_common_list1}")

# print("sbert 0.65-0.75")
# filtered_df_rag_tax = filter_by_sbert(df_rag, 'taxonomy_similarity_sbert', 0.85, 0.9)
# filtered_df_norag_tax = filter_by_sbert(df_norag, 'taxonomy_similarity_sbert', 0.85, 0.9)
# filtered_df_rag_seq = filter_by_sbert(df_rag, 'sequence_similarity_sbert', 0.85, 0.9)
# filtered_df_norag_seq = filter_by_sbert(df_norag, 'sequence_similarity_sbert', 0.85, 0.9)

# common_rag = list(set(list(filtered_df_rag_tax['source_uid'])) & set(list(filtered_df_rag_seq['source_uid'])))
# common_norag = list(set(list(filtered_df_norag_tax['source_uid'])) & set(list(filtered_df_norag_seq['source_uid'])))
# more_common_list2 = list(set(common_rag) & set(common_norag))
# print(f"{len(common_rag)} {common_rag}")
# print(f"{len(common_norag)} {common_norag}")
# print(f"{len(more_common_list2)} {more_common_list2}")

# print("sbert 0.5-0.5")
# filtered_df_rag_tax = filter_by_sbert(df_rag, 'taxonomy_similarity_sbert', 0.8,0.85)
# filtered_df_norag_tax = filter_by_sbert(df_norag, 'taxonomy_similarity_sbert', 0.8,0.85)
# filtered_df_rag_seq = filter_by_sbert(df_rag, 'sequence_similarity_sbert', 0.8,0.85)
# filtered_df_norag_seq = filter_by_sbert(df_norag, 'sequence_similarity_sbert', 0.8,0.85)

# common_rag = list(set(list(filtered_df_rag_tax['source_uid'])) & set(list(filtered_df_rag_seq['source_uid'])))
# common_norag = list(set(list(filtered_df_norag_tax['source_uid'])) & set(list(filtered_df_norag_seq['source_uid'])))
# more_common_list3 = list(set(common_rag) & set(common_norag))
# print(f"{len(common_rag)} {common_rag}")
# print(f"{len(common_norag)} {common_norag}")
# print(f"{len(more_common_list3)} {more_common_list3}")
# # find uid that made through both booleans


568 555 202 121
568 525 177 109
44
idx
idx: 13, 0.75, source_uid: 026dac2d-2ab3-4f9c-9e1d-6198db4fb080, target_uid: 2bc7d6fa-a02e-4367-b316-d6b4e8a2ce3f
idx: 28, 0.50, source_uid: 14bcb17c-f70a-41d5-b10d-294388084dfc, target_uid: 2da5c1ee-bd40-406d-83a7-2f3d93293949
idx: 34, 0.44, source_uid: 1a894d3c-b3ef-448a-a3de-2b38677cef36, target_uid: 2f46d1e6-2a85-4d46-b955-10c2eded661c
idx: 71, 0.35, source_uid: 2978ddbc-cdc9-4bfa-9a7c-4bf056904010, target_uid: 35080724-6604-401c-8b06-19b7cece3d45
idx: 113, 0.50, source_uid: 2f46d1e6-2a85-4d46-b955-10c2eded661c, target_uid: 487d752c-6e22-43e3-9c08-627bc2a6c6d4
idx: 114, 0.40, source_uid: 2f46d1e6-2a85-4d46-b955-10c2eded661c, target_uid: 487d752c-6e22-43e3-9c08-627bc2a6c6d4
idx: 213, 0.71, source_uid: 543e4c99-5d9f-407d-be75-c397d633fe56, target_uid: 737e9619-7768-407c-8a4f-6fe1e8d61f04
idx: 214, 0.71, source_uid: 543e4c99-5d9f-407d-be75-c397d633fe56, target_uid: 737e9619-7768-407c-8a4f-6fe1e8d61f04
idx: 229, 0.57, source_uid: 56fe0c73-77c4-40d

In [149]:
%history

%matplotlib qt
import sys
import os
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import mplcursors
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from bert_score import score

import pickle

sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))
from util import util_constants
from util import util_funcs
import f4_evaluate.evaluate_scene as evaluate_scene
import f1_init.database_init as database_init
import f1_init.agent_init as agent_init
import f1_init.constants_init as constants_init

#Computing similarity
import json
from sentence_transformers import SentenceTransformer
from bert_score import score
import numpy as np
from numpy.linalg import norm
import ast
import pandas as pd

#PATHS
PATH_CURR_FOLDER = os.path.abspath('') 
PATH_DATA_ANNOTATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_annotatio